In [1]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 

  

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J make_profiles
#SBATCH -n 1
#SBATCH --cpus-per-task=28
#SBATCH --mem=112G
#SBATCH -t 00:30:00
#SBATCH --constraint=HSW24
#SBATCH --exclusive
#SBATCH --nodes=1
JOB_ID=${SLURM_JOB_ID%;*}



/scratch/cnt0024/hmg2840/albert7a/anaconda3/bin/python -m distributed.cli.dask_worker tcp://172.30.100.2:46788 --nthreads 1 --nprocs 28 --memory-limit 4.29GB --name make_profiles--${JOB_ID}-- --death-timeout 60 --local-directory $SCRATCHDIR/daskjobqueue --interface ib0



In [28]:
cluster.scale(56) 


In [29]:
from dask.distributed import Client
client = Client(cluster)


In [30]:
cluster

In [31]:


client



Client Scheduler: tcp://172.30.100.3:41100 Dashboard: http://172.30.100.3:37981/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [32]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
import pandas as pd

today=date.today()

In [33]:
data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/'
gridfile='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/mesh_mask_eNATL60_3.6_lev1.nc4'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})

In [34]:
boxes_LS=pd.read_csv('../make_boxes/boxes_LS_1x1_eNATL60.csv',sep = '\t',index_col=0)
imin_LS=boxes_LS['imin']
imax_LS=boxes_LS['imax']
jmin_LS=boxes_LS['jmin']
jmax_LS=boxes_LS['jmax']
box_name=boxes_LS.index

In [35]:
imin_total=np.min([imin_LS,imax_LS])
imax_total=np.max([imin_LS,imax_LS])
jmin_total=np.min([jmin_LS,jmax_LS])
jmax_total=np.max([jmin_LS,jmax_LS])
print(jmin_total,imax_total,jmin_total,jmax_total)

3601 3063 3601 4264


In [36]:
print(dsgrid['nav_lon'].values[jmin_total,imin_total])
print(dsgrid['nav_lon'].values[jmax_total,imax_total])
print(dsgrid['nav_lat'].values[jmin_total,imin_total])
print(dsgrid['nav_lat'].values[jmax_total,imax_total])


-54.70615
-50.22431
54.950027
61.712086


In [37]:
def dx_var(data,e1):
    dx_var = (data.shift(x=-1) - data)/e1
    return dx_var
def dy_var(data,e2):
    dy_var = (data.shift(y=-1) - data)/e2
    return dy_var
    

In [38]:
filetyps = {'votemper' : 'gridT', 'vosaline' : 'gridS','vozocrtx' : 'gridU', 'vomecrty' : 'gridV','vovecrtz' : 'gridW'}
filedeps = {'votemper' : 'deptht','vosaline' : 'deptht','vozocrtx' : 'depthu', 'vomecrty' : 'depthv','vovecrtz':'depthw'}
filee1 = {'votemper' : 'e1t','vosaline' : 'e1t','vozocrtx' : 'e1u', 'vomecrty' : 'e1v','vovecrtz':'e1f'}


def compute_mean_dx_profile(var,date):
    filename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_'+filetyps[var]+'_'+date+'-'+date+'.nc'))
    file=filename[0]
    ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1,filedeps[var]:1})
    data=ds[str(var)]
    e1=dsgrid[str(filee1[var])][0]
    data_dx=dx_var(data,e1)
    profile=data_dx[:,:,jmin_total-10:jmax_total+10,imin_total-10:imax_total+10]
    attrs=data.attrs
    return profile,attrs 

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")


In [39]:
#56 workers => 6mn
#84 workers => 5min27
%time profile,attrs=compute_mean_dx_profile('votemper','20090701')

CPU times: user 11.2 s, sys: 420 ms, total: 11.6 s
Wall time: 13 s


In [17]:
profile.shape

(24, 300, 683, 387)

In [40]:
def write_profile_one_box(data_dx,var,date,ibox,attrs):
    box=box_name[ibox]
    profile_name='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/ANNA/LS/eNATL60'+box+'-BLBT02_y'+date[0:4]+'m'+date[4:6]+'d'+date[6:9]+'_dx'+var+'-profile.nc'  
    if not os.path.exists(profile_name):
        profile=data_dx[:,:,jmin_LS[ibox]-(jmin_total-10):jmax_LS[ibox]-(jmin_total-10),imin_LS[ibox]-(imin_total-10):imax_LS[ibox]-(imin_total-10)].mean(dim={'x','y','time_counter'})
        dataset=profile.to_dataset(name='dx'+var)
        dataset['dx'+var].attrs=attrs
        dataset['dx'+var].attrs['standard_name']='dx gradient of '+attrs['standard_name']
        dataset['dx'+var].attrs['long_name']='dx_'+attrs['long_name']
        dataset['dx'+var].attrs['units']='degC/m'        
        dataset.attrs['global_attribute']= 'x-gradient of '+var+' profile averaged over 24h and in '+box+' computed on occigen with dask-jobqueue '+str(today)
        dataset.to_netcdf(path=profile_name,mode='w')


In [41]:
%time write_profile_one_box(profile,'votemper','20090702',0,attrs)

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")


CPU times: user 1min 13s, sys: 7.97 s, total: 1min 21s
Wall time: 1min 52s


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")


In [ ]:
def compute_profile_one_box(data_dx,var,date,ibox):
    box=box_name[ibox]
    profile_name='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/ANNA/LS/eNATL60'+box+'-BLBT02_y'+date[0:4]+'m'+date[4:6]+'d'+date[6:9]+'_dx'+var+'-profile.nc'  
    if not os.path.exists(profile_name):
        profile=data_dx[:,:,jmin_LS[ibox]-(jmin_total-10):jmax_LS[ibox]-(jmin_total-10),imin_LS[ibox]-(imin_total-10):imax_LS[ibox]-(imin_total-10)].mean(dim={'x','y','time_counter'})
    return profile

In [51]:
computations=[]
for i in np.arange(60):
    computations.append(dask.delayed(write_profile_one_box(profile,'votemper','20090702',i,attrs)))
    

KilledWorker: ("('getitem-truediv-mean_chunk-3b72c4c5aa7cb23940fbe86f490fe418', 16, 190, 0, 0)", <Worker 'tcp://172.30.6.65:36176', memory: 0, processing: 22>)

In [47]:
computations

[Delayed('None-74e6dbc4-865a-4747-8ba3-00185226716c'),
 Delayed('None-41a0477a-1ad3-4bd2-8060-01fb1eb09872'),
 Delayed('None-9465c616-b39f-48f4-81db-31fe550f9b57'),
 Delayed('None-3b2b9884-0d7d-4ee5-9ae4-e1f93dc1ce33'),
 Delayed('None-f85c6926-6f26-40cf-88df-7f77d82f03d5'),
 Delayed('None-7b2d43e1-79b3-475d-9d05-b35e39e3770b'),
 Delayed('None-c2d64608-f60c-4645-bdd1-e994fe236fd0'),
 Delayed('None-899412b7-f03e-42f4-b5f7-7bb367ecb3e7'),
 Delayed('None-eab872b5-38ef-42b4-a12a-995cb76da578'),
 Delayed('None-728a66f2-2fc9-406a-9563-9198918f1a84'),
 Delayed('None-769421af-771a-4253-9c77-0f6606bff684'),
 Delayed('None-52536a74-0b4a-42e4-93fe-99e6e1a3b787'),
 Delayed('None-7592f48c-0cd1-4e4a-91d3-f60e7b85c04f'),
 Delayed('None-fedb9d9f-1e96-4569-a2c4-787fbe6aee4c'),
 Delayed('None-0b8a5bd0-6ab9-4720-8da4-312fc99d800d'),
 Delayed('None-c18b5b0b-ac64-4ba1-bbbf-ba4cd3fbd22f'),
 Delayed('None-5c5c044e-8ec5-49d1-bff7-eecc1e8866fc'),
 Delayed('None-5c311903-24dc-4bef-9a0d-835e4f8e8d7c'),
 Delayed('

In [48]:
results=dask.compute(computations)